# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [120]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [121]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("book_infos.csv", index_col=0)
df_books.head()

#df_books.availability.unique()

,title,price,availability,rating
0,A Light in the Attic,£51.77,In stock,Three
1,Tipping the Velvet,£53.74,In stock,One
2,Soumission,£50.10,In stock,One
3,Sharp Objects,£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,£54.23,In stock,Five


In [122]:
df_books

,title,price,availability,rating
0,A Light in the Attic,£51.77,In stock,Three
1,Tipping the Velvet,£53.74,In stock,One
2,Soumission,£50.10,In stock,One
3,Sharp Objects,£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,£54.23,In stock,Five
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock,One
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock,Four
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock,Five
998,1st to Die (Women's Murder Club #1),£53.98,In stock,One


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [123]:
# Vérification des types de données
df_books.dtypes



title           object
price           object
availability    object
rating          object
dtype: object

Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [124]:
# Conversion de title en chaîne de caractères
df_books["title"] = df_books["title"].astype(str)

# Vérification du type de la colonne title
print(df_books["title"].dtype)

object


Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [125]:
# Convertir la colonne price en type décimal
df_books["price"] = df_books["price"].astype(str).str.removeprefix("£")
df_books["price"] = df_books["price"].astype(float)

# Vérification du type de la colonne price
print(df_books["price"].dtype)

float64


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [126]:
# Valeurs possibles de la colonne availability
df_books["availability"] = df_books["availability"].astype("bool")
print(df_books["availability"].dtype)

bool


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [127]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    if value == "In stock":
        return True
    else:
        return False


Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [128]:
# Convertir la colonne availability en booléen (True/False)
df_books["availability"] = df_books["availability"].apply(convert_availability)

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)

bool


Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [129]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
df_books["rating"] = df_books["rating"].astype(str).str.strip().str.title()

ratings_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

df_books["rating"] = df_books["rating"].map(ratings_map).astype("Int64")

# Vérification du type de la colonne rating
print(df_books["rating"].dtype)

Int64


In [130]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
    # Conversion de title en chaîne de caractères
    df_books["title"] = df_books["title"].astype(str)
    
    # Convertir la colonne price en type décimal
    df_books["price"] = df_books["price"].astype(str).str.removeprefix("£")
    df_books["price"] = df_books["price"].astype(float)

    # Fonction pour convertir la valeur de availability en booléen
    def convert_availability(value : str) -> bool:
        if value == "In stock":
            return True
        else:
            return False

    # Convertir la colonne availability en booléen (True/False)
    df_books["availability"] = df_books["availability"].apply(convert_availability)

    df_books["rating"] = df_books["rating"].astype(str).str.strip().str.title()
    ratings_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    df_books["rating"] = df_books["rating"].map(ratings_map).astype("Int64")

    return df_books

print(convert_types(df_books).dtypes)

title            object
price           float64
availability       bool
rating            Int64
dtype: object


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [131]:
# Création de la BDD et insertion des données

# Création de la BDD
connection = sqlite3.connect("/home/emese/Briefs_test/scraping/book_store.db")
# On vérifie que la BDD est créé
print(connection.total_changes)

# On crée un table dans la BDD avec le df
df_books.to_sql('book_store', connection, if_exists='replace')

0


1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [132]:
# Compter le nombre de livre dans la BDD

# On crée un 'cursor' (integré dans la bibliothèque) pour interagir avec la BD
cursor = connection.cursor()

# On execute la requête pour compter le nombre de livres dans la BD
cursor.execute("SELECT COUNT(*) FROM book_store")

#On affiche du résultat
print(cursor.fetchone()[0])

#On déconnecte
#connection.close()

1000
